# Statistics and Analysis of Characters and Anime 

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import datetime
import ast

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler

import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

In [2]:
# Characters who appear >10 times in the fanfiction, their names gathered into a in a csv file. Converted to pandas DF.
characters = pd.read_csv('myhero_names_rev.csv')

# Archive of Our Own
AO3 = pd.read_csv('AO3.csv')

# Series overview: Original season number, Number of episodes, First aired, and Last aired.
og_season_anime_airdate = pd.read_csv('og_air_clean.csv')

# Season 1 Episode number overall, Episode number in the season, Title, Storyboarded by, Directed by,
#     Written by, Original air date, and English air date.
season_one_anime = pd.read_csv('season_one_clean.csv')

# Season 2-6 Episode number overall, Episode number in the season, Title, Storyboarded by, Directed by,
#     Written by, Viewership rating, Original air date, and English air date.
season_two_anime = pd.read_csv('season_two_clean.csv')
season_three_anime = pd.read_csv('season_three_clean.csv')
season_four_anime = pd.read_csv('season_four_clean.csv')
season_five_anime = pd.read_csv('season_five_clean.csv')
season_six_anime = pd.read_csv('season_six_clean.csv')

# Original Video Animation (OVA) titled "Save! Rescue Training!"
# Number (8 total released), Title, Storyboard by, Directed by, Written by, Original release date.
og_vid_animation_anime =  pd.read_csv('OVAs_clean.csv')

# Japan Home Video Release
# Season, Volume, Epsiodes, Release date.
Japan_homevid_release_anime = pd.read_csv('Japan_HV_clean.csv')

# Funimation/Crunchyroll (North America)
# Season, Volume, Episodes, Release date.
funimation_crunchyroll_anime = pd.read_csv('Fun_Crunch_NA_clean.csv')

# Universal Pictures/Manga Entertainment (UK and Ireland)
# Season, Volume, Episodes, Release date.
universalpictures_manga_anime = pd.read_csv('Up_Uk_clean.csv')

# Universal Sony/Madman Entertainment (Australia and New Zealand)
# Season, Volume, Episodes, Release date Universal, Release date Madman.
universalsony_madman_anime = pd.read_csv('Us_Me_clean.csv')

In [3]:
name_1 = [x for x in characters['1'].values]
name_2 = [x for x in (characters['2'].dropna().values)]
mhanames = name_1 + name_2


In [4]:
dont_want = ['Parents', 'Suffering', 'Loss Of His Will', 'Niece', 'mentioned', 'Family', 'Notebooks', 'Character', 'quirk', 'Friend', 'Eriks', 'Werewolf', 'Prototype', 'Mother', 'Sibling', 'Sidekicks', 'l Jade', 'Nezuko']

In [5]:
AO3['characters'] = AO3['characters'].apply(ast.literal_eval)

In [6]:
AO3['characters']

0                 [Dabi | Todoroki Touya, Kochou Shinobu]
1       [Midoriya Izuku, Dazai Osamu (Bungou Stray Dog...
2       [Power (Chainsaw Man), Ashido Mina, Iida Tenya...
3       [Midoriya Izuku, Midoriya Inko, Bakugou Katsuk...
4       [Bakugou Katsuki, Uraraka Ochako, Midoriya Izuku]
                              ...                        
1995    [Midoriya Izuku, Class 1-A (My Hero Academia),...
1996                    [Bakugou Katsuki, Midoriya Izuku]
1997    [Midoriya Izuku, Yagi Toshinori | All Might, T...
1998    [Mia Ikati, Rue Achlys, Ryuuji Takao, Itsumi M...
1999    [Midoriya Izuku, Sero Hanta, Bakugou Katsuki, ...
Name: characters, Length: 2000, dtype: object

In [7]:
for i, x in enumerate(AO3['characters']):
    names = []
    for xx in x:
        for n in mhanames:
            if n in xx:
                if xx not in names:
                    names.append(xx)
        for name in dont_want:
            if name in xx:
                if xx in names:
                    names.remove(xx)

    AO3['characters'][i] = names # Replace the original string with the new list
    

/var/folders/2h/nmfrxbhx4ts84jv_xc8vhsvr0000gn/T/ipykernel_15317/448749779.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AO3['characters'][i] = names # Replace the original string with the new list


In [8]:
AO3.shape

(2000, 7)

In [9]:
AO3.explode('characters')['characters'].value_counts()

Midoriya Izuku                1306
Bakugou Katsuki               1052
Aizawa Shouta | Eraserhead     813
Todoroki Shouto                630
Yagi Toshinori | All Might     521
                              ... 
Mei Hatsume                      1
Denki Kaminari                   1
Ochaco Uraraka                   1
Eric Cartman                     1
Kurama | Minamino Shuuichi       1
Name: characters, Length: 150, dtype: int64

In [10]:
AO3

,title,mixed fandom,ships,characters,hits,kudos,published
0,Purple poison flames meet Blue fire,['Fandoms:\n鬼滅の刃 | Demon Slayer: Kimetsu no Ya...,['Dabi | Todoroki Touya & Kochou Shinobu'],[Dabi | Todoroki Touya],0,NaN,02 Apr 2023
1,Stuck With Babysitter Duty,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,['Dazai Osamu/Nakahara Chuuya (Bungou Stray Do...,"[Midoriya Izuku, Bakugou Katsuki, Todoroki Sho...",0,NaN,02 Apr 2023
2,Power in UA!!!,"['Fandoms:\nChainsaw Man (Anime), 僕のヒーローアカデミア ...","['Bakugou Katsuki/Power', 'Power & Class 1-A']","[Ashido Mina, Iida Tenya, Aizawa Shouta | Eras...",508,7.0,02 Apr 2023
3,Balance and Bondeds,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,['Aizawa Shouta | Eraserhead/Yamada Hizashi | ...,"[Midoriya Izuku, Midoriya Inko, Bakugou Katsuk...",11924,481.0,02 Apr 2023
4,Can't Hold Our Babies,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,['Bakugou Katsuki/Midoriya Izuku/Uraraka Ochak...,"[Bakugou Katsuki, Uraraka Ochako, Midoriya Izuku]",0,NaN,02 Apr 2023
...,...,...,...,...,...,...,...
1995,Seeing double,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,"['Midoriya Izuku/Usagiyama Rumi | Miruko', 'Mi...",[Midoriya Izuku],14960,507.0,28 Mar 2023
1996,My Idol,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,['Bakugou Katsuki/Midoriya Izuku'],"[Bakugou Katsuki, Midoriya Izuku]",3348,234.0,28 Mar 2023
1997,Woodland Paths,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,['Midoriya Izuku & Yagi Toshinori | All Might'...,"[Midoriya Izuku, Yagi Toshinori | All Might, T...",4604,244.0,28 Mar 2023
1998,Boku No Hero Academia: Ones Way,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,[],"[Midoriya Izuku, Bakugou Katsuki, Todoroki Sho...",44,NaN,28 Mar 2023


In [11]:
characters['First'] = characters['1']
characters = characters.set_index('First')

In [12]:
def topchar(char, df):
    named = []
    for i in characters.loc[char]:
        if type(i) != float:
            named.append(i)
    df[char] = 0 
    for index, row in df.iterrows():
        for name in named:
            if row[char] == 0:
                if name in ''.join(row['characters']):
                    df.loc[index, char] = 1


In [13]:
topchar('Deku', AO3)
topchar('Katsuki', AO3)
topchar('Aizawa', AO3)
topchar('Todoroki Sho', AO3)
topchar('All Might', AO3)
topchar('Uraraka', AO3)
topchar('Shinsou Hitoshi', AO3)
topchar('Denki', AO3)
topchar('Inko', AO3)
topchar('Yamada Hizashi', AO3)
topchar('Dabi', AO3)
topchar('Mina', AO3)
topchar('Shigaraki Tomura', AO3)
topchar('Iida', AO3)
topchar('Keigo', AO3)

In [14]:
AO3 = AO3.rename(columns = {'Katsuki': 'Bakugo', 'Todoroki Sho': 'Shoto', 'Shinsou Hitoshi': 'Shinsou', 'Yamada Hizashi': 'Prez Mic'})

In [15]:
AO3.head()

,title,mixed fandom,ships,characters,hits,kudos,published,Deku,Bakugo,Aizawa,...,Uraraka,Shinsou,Denki,Inko,Prez Mic,Dabi,Mina,Shigaraki Tomura,Iida,Keigo
0,Purple poison flames meet Blue fire,['Fandoms:\n鬼滅の刃 | Demon Slayer: Kimetsu no Ya...,['Dabi | Todoroki Touya & Kochou Shinobu'],[Dabi | Todoroki Touya],0,NaN,02 Apr 2023,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,Stuck With Babysitter Duty,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,['Dazai Osamu/Nakahara Chuuya (Bungou Stray Do...,"[Midoriya Izuku, Bakugou Katsuki, Todoroki Sho...",0,NaN,02 Apr 2023,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,Power in UA!!!,"['Fandoms:\nChainsaw Man (Anime), 僕のヒーローアカデミア ...","['Bakugou Katsuki/Power', 'Power & Class 1-A']","[Ashido Mina, Iida Tenya, Aizawa Shouta | Eras...",508,7.0,02 Apr 2023,0,1,1,...,1,0,1,0,0,0,1,0,1,0
3,Balance and Bondeds,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,['Aizawa Shouta | Eraserhead/Yamada Hizashi | ...,"[Midoriya Izuku, Midoriya Inko, Bakugou Katsuk...",11924,481.0,02 Apr 2023,1,1,1,...,0,1,0,1,1,0,0,0,0,0
4,Can't Hold Our Babies,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,['Bakugou Katsuki/Midoriya Izuku/Uraraka Ochak...,"[Bakugou Katsuki, Uraraka Ochako, Midoriya Izuku]",0,NaN,02 Apr 2023,1,1,0,...,1,0,0,0,0,0,0,0,0,0
